In this notebook, we are cleaning the 311 data from 2017 to 2024, year by year, due to the large size of each raw file. \
Each raw file was donwloaded from https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/about_data. \
The query was set the created_date with each eary, and closed_date between the start of the year and the end of the second year. These query criteria has proven to be a very generous appraoch to include as many as cases that were open longer than usual. It was later discovered that on average, most cases closed within a week, and 94~95% closed within 90 days. \
We will then concatenate these cleaned and reduced df together to form the final 311_df in the next notebook.  \
We will further use the merged 311_df to merge with eviction data. \
Finally, we will perform analysis on 311 complaint data and eviction data.  \

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns

# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data source:
# very very very large files, have to save after each cleanning
# do one file at a time
file_path1 = '/content/drive/My Drive/X999/311_different_years/2017-2018_311.csv'# 2017, 2018 both in it
file_path2 = '/content/drive/My Drive/X999/311_different_years/2019_311.csv'# 2019
file_path3 = '/content/drive/My Drive/X999/311_different_years/2020_311.csv'# mostly 2020, some 2017-2019 ones will be removed
file_path4 = '/content/drive/My Drive/X999/311_different_years/2021_311.csv'# 2021
file_path5 = '/content/drive/My Drive/X999/311_different_years/2022_311.csv'# 2022
file_path6 = '/content/drive/My Drive/X999/311_different_years/2023_311.csv'# 2023
file_path7 = '/content/drive/My Drive/X999/311_different_years/2024_311.csv'# 2024

In [ ]:
complaints_df = pd.read_csv(file_path1)
# complaints_df_2 = pd.read_csv(file_path8)

In [ ]:
# copy_df = complaints_df.copy()

In [ ]:
# combined_df = pd.concat([complaints_df, complaints_df_2], ignore_index=True)

In [ ]:
# compliants_df = combined_df.copy()

In [ ]:
complaints_df.columns, complaints_df.shape

(Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
        'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
        'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
        'Intersection Street 1', 'Intersection Street 2', 'Address Type',
        'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
        'Resolution Description', 'Resolution Action Updated Date',
        'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
        'Y Coordinate (State Plane)', 'Open Data Channel Type',
        'Park Facility Name', 'Park Borough', 'Vehicle Type',
        'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
        'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
        'Latitude', 'Longitude', 'Location'],
       dtype='object'),
 (5265027, 41))

In [ ]:
complaints_df.shape

(5265027, 41)

In [ ]:
# cleaning functions

# some cleaning ideas are from:
# https://www.kaggle.com/code/riyapatel1697/nyc-311-customer-service-requests-analysis

def drop_columns(df, columns):
    df.drop(columns, axis=1, inplace=True)

def clean_column_names(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.strip().str.replace(' ', '_')
    return df

# drop columns that are empty or almost empty
def drop_empty_columns(df):
    # drop columns that are completely or almost empty
    # and get the count of NaNs in each column
    na_cololumn_count = df.isna().sum()
    columns_to_drop = []
    # use .items() instead of .iteritems()
    for column, na_count in na_cololumn_count.items():
        if na_count / len(df) >= 0.85:
            columns_to_drop.append(column)

    df.drop(columns_to_drop, axis=1, inplace=True)
    return df

def drop_unneccessary_columns(df):
    columns_to_drop = ['x_coordinate_(state_plane)', 'agency_name', 'agency',
                       'y_coordinate_(state_plane)', 'open_data_channel_type',
                       'park_facility_name', 'park_borough', 'vehicle_type',
                       'due_date', 'taxi_company_borough', 'taxi_pick_up_location',
                       'bridge_highway_name', 'bridge_highway_direction', 'road_ramp',
                       'bridge_highway_segment', 'facility_type', "community_board", "landmark"]

    df_cleaned = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

    return df_cleaned

def clean_date_times(df, created_date='created_date', closed_date='closed_date',
                     cutoff_date='2024-12-31 23:59:59'):
    # step by step reducing and return a new dataframe
    original_shape = df.shape
    print(f"filtered_df shape: {original_shape}")
    cutoff_date = pd.Timestamp(cutoff_date)
    min_plausible_date = pd.Timestamp('2017-01-01')
    condition1 = df[closed_date] >= df[created_date]
    filtered1 = df[condition1]
    print(f"after closed ≥ created: {filtered1.shape}")
    condition2 = df[closed_date] <= cutoff_date
    filtered2 = df[condition2]
    print(f"after closed ≤ cutoff: {filtered2.shape}")
    condition3 = df[closed_date] >= min_plausible_date
    filtered3 = df[condition3]
    print(f"after closed ≥ min_plausible: {filtered3.shape}")
    final_df = df[condition1 & condition2 & condition3]
    final_df = final_df.copy()
    print(f"cleaned filtered dataframe shape: {final_df.shape}")
    return final_df

def get_whole_year_cases(year, df, created_col='created_date', closed_col='closed_date'):

    # this function works on one or multiple dataframes so that we can accurate get the data from
    # strictly just one year with the cut-off date for close_date that is 90 days later after the year ends

    created_start = pd.Timestamp(f'{year}-01-01 00:00:00')
    created_end = pd.Timestamp(f'{year}-12-31 23:59:59')

    # set closure cutoff to March 31st of the following year (90 days), because 94~95% of the cases close
    # within 90 days and we just want the majority of the cases created within a certain whole year
    closed_cutoff = pd.Timestamp(f'{year+1}-03-31 23:59:59')
    records_year = df[(df[created_col] >= created_start) &
                      (df[created_col] <= created_end) &
                      (df[closed_col] >= created_start) &
                      (df[closed_col] <= closed_cutoff)]

    total_created = len(df[(df[created_col] >= created_start) &
                          (df[created_col] <= created_end)])

    print(f"total records in {year}: {len(df)} cases")
    print(f"cases created in {year}: {total_created} cases")
    print(f"cases created in {year} and closed by March 31, {year+1}: {len(records_year)} cases")
    print(f"percentage of {year} cases captured within our cutoff: {(len(records_year) / total_created) * 100:.2f}%")

    return records_year

def calculate_closure_duration(df, created_col='created_date', closed_col='closed_date'):
    # added new temporary columns to make calculations easier, but will eventually get rid of them for later
    df['duration_days'] = (df[closed_col] - df[created_col]).dt.total_seconds() / (60 * 60 * 24)
    avg_duration = df['duration_days'].mean()
    median_duration = df['duration_days'].median()
    min_duration = df['duration_days'].min()
    max_duration = df['duration_days'].max()

    print(f"average closure duration: {avg_duration:.2f} days")
    print(f"median closure duration: {median_duration:.2f} days")
    print(f"minimum closure duration: {min_duration:.2f} days")
    print(f"maximum closure duration: {max_duration:.2f} days")

    print("summary:")
    bins = [0, 1, 7, 30, 90, 180, 365, float('inf')]
    labels = ['Same day', '1-7 days', '8-30 days', '31-90 days',
              '91-180 days', '181-365 days', 'Over 1 year']
    df['duration_category'] = pd.cut(df['duration_days'], bins=bins, labels=labels)

    duration_counts = df['duration_category'].value_counts().sort_index()
    for category, count in duration_counts.items():
        percentage = (count / len(df)) * 100
        print(f"{category}: {count} cases ({percentage:.2f}%)")

    return df['duration_days']

def find_majority_year_data(filtered_df, created_col='created_date', closed_col='closed_date'):
    filtered_df['created_year'] = filtered_df[created_col].dt.year
    filtered_df['closed_year'] = filtered_df[closed_col].dt.year

    created_counts = filtered_df['created_year'].value_counts().sort_index()

    closed_counts = filtered_df['closed_year'].value_counts().sort_index()

    print("distribution by created_date year:")
    for year, count in created_counts.items():
        percentage = (count / len(filtered_df)) * 100
        print(f"{year}: {count} cases ({percentage:.2f}%)")

    print("distribution by closed_date year:")
    for year, count in closed_counts.items():
        percentage = (count / len(filtered_df)) * 100
        print(f"{year}: {count} cases ({percentage:.2f}%)")

    majority_year = created_counts.idxmax()
    majority_count = created_counts.max()
    majority_percentage = (majority_count / len(filtered_df)) * 100

    print(f"\nMajority year by creation date: {majority_year} with {majority_count} cases ({majority_percentage:.2f}%)")

    return majority_year

In [ ]:
complaints_df.shape # complaints_2023.shape (3225555, 41)

(5265027, 41)

In [ ]:
# make sure the column names are all lower case with hiphen, so that it would easier for cleaning
complaints_df = clean_column_names(complaints_df)

In [ ]:
# find complaint types that have something to do with noice or constructions
noise_construction_complaints = complaints_df[complaints_df['complaint_type'].str.contains(r'(?i)noise|construction', regex=True)]
noise_construction_complaints.complaint_type.unique()

array(['General Construction/Plumbing', 'Noise', 'Noise - Commercial',
       'Noise - Residential', 'Noise - Street/Sidewalk',
       'Noise - Vehicle', 'Noise - Helicopter', 'Noise - Park',
       'Construction Safety Enforcement', 'Collection Truck Noise',
       'Noise - House of Worship'], dtype=object)

In [ ]:
complaints_df.shape

(5265027, 41)

In [ ]:
# the type of complaints we want to keep: noise related
# AND we also want to keep the location type that is family residential related.
# so this is an OR situation, we keep them both here and do deeper cleaning later.

# following the last cell
noise_complaint_types = [
    'Noise - Residential', 'Noise - Helicopter', 'Noise - House of Worship', 'Noise - Commercial', 'Noise - Park',
    'Noise - Street/Sidewalk', 'Noise', 'Noise - Vehicle',
    'General Construction/Plumbing',
    'Collection Truck Noise', 'Construction Lead Dust',
    'COVID-19 Non-essential Construction',
    'Construction Safety Enforcement'
]
# particularly picked complaint types that tend to have stronger correlations between complaints and evictions.
must_include = [
    "Noise - Residential", "Unsanitary Condition", "Heat/Hot Water", "Flooring/Stairs", "Indoor Air Quality", "Smoking", "Door/Window",
    "Paint/Plaster", "Water Leak", "Electric", "Safety", "Plumbing", "Appliance", "Rodent", "Unsanitary Pigeon Condition", "Asbestos",
    "Elevator", "Construction Lead Dust", "Indoor Sewage", "Peeling Paint", "Window Guard", "Harboring Bees/Wasps", "Drinking Water",
    "Noise - Street/Sidewalk", "Mold"]

In [ ]:
# filter rows where complaint_type is in the noise_complaint_types list or location_type contains the keywords
filtered_df = complaints_df[
    (complaints_df['complaint_type'].isin(noise_complaint_types)) |
     (complaints_df["complaint_type"].isin(must_include)) |
    (complaints_df['location_type'].str.contains(r'family|residential', case=False, na=False))
]

In [ ]:
# basic cleaning
filtered_df = drop_unneccessary_columns(filtered_df)
filtered_df = drop_empty_columns(filtered_df)

In [ ]:
filtered_df.shape

(2364600, 21)

In [ ]:
filtered_df.complaint_type.unique(), filtered_df.location_type.unique(), len(filtered_df.complaint_type.unique()), len(filtered_df.location_type.unique())

(array(['General Construction/Plumbing', 'HEAT/HOT WATER', 'DOOR/WINDOW',
        'WATER LEAK', 'UNSANITARY CONDITION', 'Vending', 'APPLIANCE',
        'PLUMBING', 'PAINT/PLASTER', 'SAFETY', 'Graffiti', 'Elevator',
        'Noise', 'Noise - Commercial', 'Homeless Person Assistance',
        'Noise - Residential', 'OUTSIDE BUILDING', 'FLOORING/STAIRS',
        'Noise - Street/Sidewalk', 'GENERAL', 'ELEVATOR',
        'DOF Property - Reduction Issue', 'Homeless Encampment',
        'Non-Residential Heat', 'Animal Abuse',
        'Non-Emergency Police Matter', 'Noise - Vehicle', 'ELECTRIC',
        'Indoor Sewage', 'Noise - Helicopter', 'Indoor Air Quality',
        'Noise - Park', 'Asbestos', 'Construction Safety Enforcement',
        'Plumbing', 'Smoking', 'Collection Truck Noise', 'Rodent',
        'Unsanitary Animal Pvt Property', 'Noise - House of Worship',
        'Mold', 'Drinking Water', 'Urinating in Public',
        'Unsanitary Pigeon Condition', 'Drinking', 'Animal-Abuse',
    

In [ ]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2364600 entries, 5 to 5265026
Data columns (total 21 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   unique_key                      int64  
 1   created_date                    object 
 2   closed_date                     object 
 3   complaint_type                  object 
 4   descriptor                      object 
 5   location_type                   object 
 6   incident_zip                    object 
 7   incident_address                object 
 8   street_name                     object 
 9   cross_street_1                  object 
 10  cross_street_2                  object 
 11  address_type                    object 
 12  city                            object 
 13  status                          object 
 14  resolution_description          object 
 15  resolution_action_updated_date  object 
 16  bbl                             float64
 17  borough                         

In [ ]:
filtered_df.head(2)

,unique_key,created_date,closed_date,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,...,address_type,city,status,resolution_description,resolution_action_updated_date,bbl,borough,latitude,longitude,location
5,41313741,12/31/2018 11:58:47 AM,01/10/2019 12:00:00 AM,General Construction/Plumbing,Building Permit - None,NaN,10451.0,825 GERARD AVENUE,GERARD AVENUE,NaN,...,ADDRESS,BRONX,Closed,The Department of Buildings investigated this ...,01/10/2019 12:00:00 AM,2.024830e+09,BRONX,40.825866,-73.925913,"(40.825865645198235, -73.92591342579004)"
6,41335801,12/31/2018 11:58:44 AM,01/03/2019 10:25:10 AM,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10458.0,2410 WASHINGTON AVENUE,WASHINGTON AVENUE,NaN,...,ADDRESS,BRONX,Closed,The Department of Housing Preservation and Dev...,01/03/2019 10:25:10 AM,2.030570e+09,BRONX,40.858053,-73.890913,"(40.858052508996636, -73.8909130551251)"


In [ ]:
filtered_df.isna().sum().where(lambda x:x>0).dropna()

,0
closed_date,13329.0
descriptor,7406.0
location_type,281354.0
incident_zip,49342.0
incident_address,119175.0
street_name,119176.0
cross_street_1,1468900.0
cross_street_2,1470038.0
address_type,45572.0
city,49015.0


In [ ]:
filtered_df = filtered_df.dropna(subset=['bbl'])
filtered_df.shape

(2124544, 21)

In [ ]:
# convert to datetime
filtered_df.created_date =  pd.to_datetime(filtered_df.created_date)
filtered_df.closed_date =  pd.to_datetime(filtered_df.closed_date)

In [ ]:
filtered_df.created_date.min(), filtered_df.created_date.max(), \
filtered_df.closed_date.min(), filtered_df.closed_date.max()

(Timestamp('2017-01-01 00:00:00'),
 Timestamp('2018-12-31 11:58:47'),
 Timestamp('1900-01-01 00:00:00'),
 Timestamp('2025-02-28 00:00:00'))

In [ ]:
duration_days = calculate_closure_duration(filtered_df)

average closure duration: 13.16 days
median closure duration: 2.63 days
minimum closure duration: -42872.51 days
maximum closure duration: 2920.40 days
summary:
Same day: 772986 cases (36.38%)
1-7 days: 683573 cases (32.18%)
8-30 days: 482064 cases (22.69%)
31-90 days: 112922 cases (5.32%)
91-180 days: 20403 cases (0.96%)
181-365 days: 5561 cases (0.26%)
Over 1 year: 7384 cases (0.35%)


In [ ]:
majority_year = find_majority_year_data(filtered_df)
# turns out this particular one, when I downloaded it, spanner over 2 years.
# all the other raw csv only contained a little bit over one year

distribution by created_date year:
2017: 1046038 cases (49.24%)
2018: 1078506 cases (50.76%)
distribution by closed_date year:
1900.0: 7 cases (0.00%)
2001.0: 1 cases (0.00%)
2016.0: 2 cases (0.00%)
2017.0: 994102 cases (46.79%)
2018.0: 1084013 cases (51.02%)
2019.0: 30317 cases (1.43%)
2020.0: 1047 cases (0.05%)
2021.0: 475 cases (0.02%)
2022.0: 326 cases (0.02%)
2023.0: 832 cases (0.04%)
2024.0: 1858 cases (0.09%)
2025.0: 126 cases (0.01%)

Majority year by creation date: 2018 with 1078506 cases (50.76%)


In [ ]:
cleaned_filtered_df = clean_date_times(filtered_df,
                                      created_date='created_date',
                                      closed_date='closed_date')

filtered_df shape: (2124544, 25)
after closed ≥ created: (2112807, 25)
after closed ≤ cutoff: (2112980, 25)
after closed ≥ min_plausible: (2113096, 25)
cleaned filtered dataframe shape: (2112681, 25)


In [ ]:
type(cleaned_filtered_df.created_date)

pandas.core.series.Series

In [ ]:
cleaned_filtered_df['created_date'].min(), cleaned_filtered_df['created_date'].max(), \
cleaned_filtered_df['closed_date'].min(), cleaned_filtered_df['closed_date'].max()
# good

(Timestamp('2017-01-01 00:00:00'),
 Timestamp('2018-12-31 11:58:47'),
 Timestamp('2017-01-01 00:00:00'),
 Timestamp('2024-12-29 00:00:00'))

In [ ]:
cleaned_filtered_df.head()

,unique_key,created_date,closed_date,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,...,resolution_action_updated_date,bbl,borough,latitude,longitude,location,duration_days,duration_category,created_year,closed_year
5,41313741,2018-12-31 11:58:47,2019-01-10 00:00:00,General Construction/Plumbing,Building Permit - None,NaN,10451.0,825 GERARD AVENUE,GERARD AVENUE,NaN,...,01/10/2019 12:00:00 AM,2.024830e+09,BRONX,40.825866,-73.925913,"(40.825865645198235, -73.92591342579004)",9.500845,8-30 days,2018,2019.0
6,41335801,2018-12-31 11:58:44,2019-01-03 10:25:10,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10458.0,2410 WASHINGTON AVENUE,WASHINGTON AVENUE,NaN,...,01/03/2019 10:25:10 AM,2.030570e+09,BRONX,40.858053,-73.890913,"(40.858052508996636, -73.8909130551251)",2.935023,1-7 days,2018,2019.0
13,41481637,2018-12-31 11:57:41,2019-01-22 02:05:22,DOOR/WINDOW,WINDOW FRAME,RESIDENTIAL BUILDING,11207.0,474 RIVERDALE AVENUE,RIVERDALE AVENUE,NaN,...,01/22/2019 02:05:22 AM,3.038340e+09,BROOKLYN,40.662480,-73.898600,"(40.662480035678755, -73.89860033599884)",21.588669,8-30 days,2018,2019.0
14,41484466,2018-12-31 11:57:41,2019-01-22 02:05:22,WATER LEAK,HEAVY FLOW,RESIDENTIAL BUILDING,11207.0,474 RIVERDALE AVENUE,RIVERDALE AVENUE,NaN,...,01/22/2019 02:05:22 AM,3.038340e+09,BROOKLYN,40.662480,-73.898600,"(40.662480035678755, -73.89860033599884)",21.588669,8-30 days,2018,2019.0
15,41845281,2018-12-31 11:57:41,2019-03-01 09:47:08,UNSANITARY CONDITION,MOLD,RESIDENTIAL BUILDING,11378.0,59-15 MASPETH AVENUE,MASPETH AVENUE,NaN,...,03/01/2019 09:47:08 AM,4.026910e+09,QUEENS,40.723177,-73.907819,"(40.72317740723669, -73.90781948052384)",59.909340,31-90 days,2018,2019.0


In [ ]:
filtered_df.shape, cleaned_filtered_df.shape # ((2124544, 21), (2112681, 21)), ((1046019, 21), (1038184, 21))

((2124544, 25), (2112681, 25))

In [ ]:
records_2017 = get_whole_year_cases(2017, cleaned_filtered_df)

total records in 2017: 2112681 cases
cases created in 2017: 1038203 cases
cases created in 2017 and closed by March 31, 2018: 1032946 cases
percentage of 2017 cases captured within our cutoff: 99.49%


In [ ]:
records_2018 = get_whole_year_cases(2018, cleaned_filtered_df)
# one thing about this particular function is, it works on any one or multiple dataframes to get just one year's data

total records in 2018: 2112681 cases
cases created in 2018: 1074478 cases
cases created in 2018 and closed by March 31, 2019: 1066249 cases
percentage of 2018 cases captured within our cutoff: 99.23%


In [ ]:
records_2017.created_date.min(), records_2017.created_date.max(), \
records_2017.closed_date.min(),records_2017.closed_date.max(), \
records_2017.shape

(Timestamp('2017-01-01 00:00:00'),
 Timestamp('2017-12-31 23:59:35'),
 Timestamp('2017-01-01 00:00:00'),
 Timestamp('2018-03-31 08:11:33'),
 (1032946, 25))

In [ ]:
records_2018.created_date.min(), records_2018.created_date.max(), \
records_2018.closed_date.min(),records_2018.closed_date.max(), \
records_2018.shape

(Timestamp('2018-01-01 00:00:00'),
 Timestamp('2018-12-31 11:58:47'),
 Timestamp('2018-01-01 00:00:00'),
 Timestamp('2019-03-31 18:10:54'),
 (1066249, 25))

In [ ]:
# due to very large file size, save every year's cleaned version to the drive and then later merge them
# do it one by one later
records_2017.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2017.csv', index=False)

In [ ]:
records_2018.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2017.csv', index=False)

## After first round of cleanning and reducing, we realize that we need to further reduce the size by removing some more irrelevant columns

In [ ]:
# get them back from the cloud
saved_2017 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2017.csv"
saved_2018 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2018.csv"
saved_2019 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2019.csv"
saved_2020 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2020.csv"
saved_2021 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2021.csv"
saved_2022 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2022.csv"
saved_2023 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2023.csv"
saved_2024 = "/content/drive/My Drive/X999/311_different_years/filtered_df_2024.csv"

In [ ]:
df_2017 = pd.read_csv(saved_2017)
df_2018 = pd.read_csv(saved_2018)
df_2019 = pd.read_csv(saved_2019)
df_2020 = pd.read_csv(saved_2020)
df_2021 = pd.read_csv(saved_2021)
df_2022 = pd.read_csv(saved_2022)
df_2023 = pd.read_csv(saved_2023)
df_2024 = pd.read_csv(saved_2024)

In [ ]:
df_2017.head()

,unique_key,created_date,closed_date,complaint_type,descriptor,location_type,incident_zip,incident_address,city,status,bbl,borough,latitude,longitude,duration_days,duration_category,created_year,closed_year
0,38070156,2017-12-31 23:59:35,2018-01-04 19:27:02,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10030.0,181 WEST 135 STREET,NEW YORK,Closed,1.019200e+09,MANHATTAN,40.815127,-73.943252,3.810729,1-7 days,2017,2018.0
1,38067146,2017-12-31 23:59:34,2018-01-01 00:57:19,Noise - Residential,Loud Talking,Residential Building/House,10035.0,2048 MADISON AVENUE,NEW YORK,Closed,1.017540e+09,MANHATTAN,40.808655,-73.938532,0.040104,Same day,2017,2018.0
2,38066214,2017-12-31 23:59:15,2018-01-01 02:48:23,Noise - Residential,Loud Music/Party,Residential Building/House,10466.0,1902 NEREID AVENUE,BRONX,Closed,2.050540e+09,BRONX,40.898700,-73.848528,0.117454,Same day,2017,2018.0
3,38067041,2017-12-31 23:58:38,2018-01-01 02:53:28,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11230.0,1201 AVENUE H,BROOKLYN,Closed,3.066870e+09,BROOKLYN,40.629675,-73.964939,0.121412,Same day,2017,2018.0
4,38068229,2017-12-31 23:58:33,2018-01-08 13:30:58,HEAT/HOT WATER,APARTMENT ONLY,RESIDENTIAL BUILDING,11226.0,70 LINDEN BOULEVARD,BROOKLYN,Closed,3.050860e+09,BROOKLYN,40.652289,-73.956328,7.564178,8-30 days,2017,2018.0


In [ ]:
additional_irrelevant_columns = [
    'street_name',
    'cross_street_1',
    'cross_street_2',
    'resolution_description',
    'resolution_action_updated_date',
    'location',
    'address_type'
]

In [ ]:
df_2017.columns

Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
       'descriptor', 'location_type', 'incident_zip', 'incident_address',
       'street_name', 'cross_street_1', 'cross_street_2', 'address_type',
       'city', 'status', 'resolution_description',
       'resolution_action_updated_date', 'bbl', 'borough', 'latitude',
       'longitude', 'location', 'duration_days', 'duration_category',
       'created_year', 'closed_year'],
      dtype='object')

In [ ]:
df_2017 = df_2017.drop(columns=additional_irrelevant_columns)
df_2018 = df_2018.drop(columns=additional_irrelevant_columns)
df_2019 = df_2019.drop(columns=additional_irrelevant_columns)
df_2020 = df_2020.drop(columns=additional_irrelevant_columns)
df_2021 = df_2021.drop(columns=additional_irrelevant_columns)
df_2022 = df_2022.drop(columns=additional_irrelevant_columns)
df_2023 = df_2023.drop(columns=additional_irrelevant_columns)
df_2024 = df_2024.drop(columns=additional_irrelevant_columns)

In [ ]:
df_2019.columns

Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
       'descriptor', 'location_type', 'incident_zip', 'incident_address',
       'intersection_street_1', 'intersection_street_2', 'city', 'status',
       'bbl', 'borough', 'latitude', 'longitude', 'duration_days',
       'duration_category', 'created_year', 'closed_year'],
      dtype='object')

In [ ]:
df_2024.columns

Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
       'descriptor', 'location_type', 'incident_zip', 'incident_address',
       'intersection_street_1', 'intersection_street_2', 'city', 'status',
       'bbl', 'borough', 'latitude', 'longitude', 'duration_days',
       'duration_category', 'created_year', 'closed_year'],
      dtype='object')

In [ ]:
df_2017.columns

Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
       'descriptor', 'location_type', 'incident_zip', 'incident_address',
       'city', 'status', 'bbl', 'borough', 'latitude', 'longitude',
       'duration_days', 'duration_category', 'created_year', 'closed_year'],
      dtype='object')

In [ ]:
df_2017.shape, df_2018.shape, df_2019.shape, df_2020.shape, df_2021.shape, df_2022.shape, df_2023.shape, df_2024.shape

((1032946, 18),
 (1066249, 18),
 (996996, 20),
 (1263463, 18),
 (1355688, 20),
 (1433295, 18),
 (1412135, 18),
 (1527906, 20))

In [ ]:
additional_irrelevant_columns2 = [
    'intersection_street_1',
    'intersection_street_2',
]

In [ ]:
df_2020 = df_2020.drop(columns=additional_irrelevant_columns2)
df_2022 = df_2022.drop(columns=additional_irrelevant_columns2)
df_2023 = df_2023.drop(columns=additional_irrelevant_columns2)

In [ ]:
df_2024 = df_2024.drop(columns=additional_irrelevant_columns2)
df_2021 = df_2021.drop(columns=additional_irrelevant_columns2)
df_2019 = df_2019.drop(columns=additional_irrelevant_columns2)

In [ ]:
df_2017.shape, df_2018.shape, df_2019.shape, df_2020.shape, df_2021.shape, df_2022.shape, df_2023.shape, df_2024.shape

((1032946, 18),
 (1066249, 18),
 (996996, 18),
 (1263463, 18),
 (1355688, 18),
 (1433295, 18),
 (1412135, 18),
 (1527906, 18))

In [ ]:
# slimmer and more sensile version
df_2017.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2017.csv', index=False)
df_2018.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2018.csv', index=False)
df_2019.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2019.csv', index=False)
df_2020.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2020.csv', index=False)
df_2021.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2021.csv', index=False)
df_2022.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2022.csv', index=False)
df_2023.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2023.csv', index=False)
df_2024.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2024.csv', index=False)

In [ ]:
df_2017.columns, df_2017.shape

(Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
        'descriptor', 'location_type', 'incident_zip', 'incident_address',
        'city', 'status', 'bbl', 'borough', 'latitude', 'longitude',
        'duration_days', 'duration_category', 'created_year', 'closed_year'],
       dtype='object'),
 (1032946, 18))

In [ ]:
df_2017.location_type.unique()

array(['RESIDENTIAL BUILDING', 'Residential Building/House',
       'Street/Sidewalk', 'Store/Commercial', 'Club/Bar/Restaurant', nan,
       'Residential Building', '3+ Family Apartment Building',
       'Residential', 'Other (Explain Below)', 'Commercial Building',
       '3+ Family Apt. Building', '1-2 Family Dwelling',
       '3+ Family Mixed Use Building', '1-2 Family Mixed Use Building',
       'Vacant Building', '1-3 Family Dwelling', 'House of Worship',
       'Sidewalk', 'Above Address', 'Construction Site', 'Public Garden',
       'Single Room Occupancy (SRO)', 'School/Pre-School', 'Restaurant',
       'Parking Lot/Garage', 'Hospital', 'Vacant Lot', 'School',
       'Office Building', 'Park/Playground', 'Government Building',
       'Catch Basin/Sewer', 'Public Stairs', 'Day Care/Nursery',
       '1-3 Family Mixed Use Building', 'Summer Camp',
       'Residential Property'], dtype=object)

In [ ]:
df_2017.incident_address.unique()

array(['181 WEST  135 STREET', '2048 MADISON AVENUE',
       '1902 NEREID AVENUE', ..., '435 WILLOWBROOK ROAD',
       '74 EAST   95 STREET', '520 EAST  144 STREET'], dtype=object)

## However, we could even further reduce the dimension of the dataframe, since they are still very large and contain some less important featurees. To make the future concatenating with each other or merging with other types of datasets more seamless, we decide to further reduce the dataframe dimension.

In [ ]:
additional_irrelevant_columns3 = [
    'descriptor',
    'location_type',
    'status',
    'city',
    'created_year',
    'closed_year',
    'duration_days',
    'duration_category'
]

In [ ]:
df_2017 = df_2017.drop(columns=additional_irrelevant_columns3)
df_2018 = df_2018.drop(columns=additional_irrelevant_columns3)
df_2019 = df_2019.drop(columns=additional_irrelevant_columns3)
# df_2020 = df_2020.drop(columns=additional_irrelevant_columns3)
df_2021 = df_2021.drop(columns=additional_irrelevant_columns3)
# df_2022 = df_2022.drop(columns=additional_irrelevant_columns3)
# df_2023 = df_2023.drop(columns=additional_irrelevant_columns3)
df_2024 = df_2024.drop(columns=additional_irrelevant_columns3)

In [ ]:
additional_irrelevant_columns3 = [
    'descriptor',
    'location_type',
    # 'incident_address',
    'status',
    'city',
    'duration_days',
    'duration_category'
]

In [ ]:
df_2020 = df_2020.drop(columns=additional_irrelevant_columns3)
df_2022 = df_2022.drop(columns=additional_irrelevant_columns3)
df_2023 = df_2023.drop(columns=additional_irrelevant_columns3)

In [ ]:
len(df_2020.columns)

12

In [ ]:
df_2017.shape, df_2018.shape, df_2019.shape, df_2020.shape, df_2021.shape, df_2022.shape, df_2023.shape, df_2024.shape

((1032946, 10),
 (1066249, 10),
 (996996, 10),
 (1263463, 10),
 (1355688, 10),
 (1433295, 10),
 (1412135, 10),
 (1527906, 10))

In [ ]:
len(df_2024.columns)

10

In [ ]:
df_2017.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2017_reduced.csv', index=False)
df_2018.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2018_reduced.csv', index=False)
df_2019.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2019_reduced.csv', index=False)
df_2020.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2020_reduced.csv', index=False)
df_2021.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2021_reduced.csv', index=False)
df_2022.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2022_reduced.csv', index=False)
df_2023.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2023_reduced.csv', index=False)
df_2024.to_csv('/content/drive/My Drive/X999/311_different_years/filtered_df_2024_reduced.csv', index=False)

In [ ]:
link = '/content/drive/My Drive/X999/311_different_years/filtered_df_2024_reduced.csv'

In [ ]:
df = pd.read_csv(link)

In [ ]:
len(df.columns)

10

In [ ]:
# a current and final look (we may even futher reduce them during analysis)
# 10 columns in total
df_2017

,unique_key,created_date,closed_date,complaint_type,incident_zip,incident_address,bbl,borough,latitude,longitude
0,38070156,2017-12-31 23:59:35,2018-01-04 19:27:02,HEAT/HOT WATER,10030.0,181 WEST 135 STREET,1.019200e+09,MANHATTAN,40.815127,-73.943252
1,38067146,2017-12-31 23:59:34,2018-01-01 00:57:19,Noise - Residential,10035.0,2048 MADISON AVENUE,1.017540e+09,MANHATTAN,40.808655,-73.938532
2,38066214,2017-12-31 23:59:15,2018-01-01 02:48:23,Noise - Residential,10466.0,1902 NEREID AVENUE,2.050540e+09,BRONX,40.898700,-73.848528
3,38067041,2017-12-31 23:58:38,2018-01-01 02:53:28,Noise - Street/Sidewalk,11230.0,1201 AVENUE H,3.066870e+09,BROOKLYN,40.629675,-73.964939
4,38068229,2017-12-31 23:58:33,2018-01-08 13:30:58,HEAT/HOT WATER,11226.0,70 LINDEN BOULEVARD,3.050860e+09,BROOKLYN,40.652289,-73.956328
...,...,...,...,...,...,...,...,...,...,...
1032941,35147286,2017-01-01 00:00:00,2017-01-01 00:00:00,HEAT/HOT WATER,10454.0,520 EAST 144 STREET,2.022690e+09,BRONX,40.811424,-73.916165
1032942,35147155,2017-01-01 00:00:00,2017-01-04 16:19:56,Unsanitary Pigeon Condition,11208.0,735 LINCOLN AVENUE,3.042719e+09,BROOKLYN,40.671189,-73.866071
1032943,35146489,2017-01-01 00:00:00,2017-01-05 00:00:00,Rodent,10028.0,315 EAST 86 STREET,1.015490e+09,MANHATTAN,40.777632,-73.951144
1032944,35147048,2017-01-01 00:00:00,2017-01-01 00:00:00,Rodent,10003.0,71 EAST 4 STREET,1.004600e+09,MANHATTAN,40.726604,-73.990381
